# Importing Libraries

In [1]:
import pyrootutils
import sys

root = pyrootutils.setup_root(
    search_from=sys.path[0],
    pythonpath=True,
    cwd=True,
)

import torch
import torchvision
from core_utils import main
from core_utils.utils import data_handling, train, test, gradcam, helpers, augmentation
from core_utils.models import resnet, s8_custom_resnet
from pprint import pprint
# from torch_lr_finder import LRFinder


import timm
import urllib
import torch
import os
import numpy as np

from torch import nn
import torchvision.transforms as T
import torch.nn.functional as F

from PIL import Image
from S9.datamodule import CIFARDataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, Trainer, seed_everything
from matplotlib.colors import LinearSegmentedColormap
from torch.optim.lr_scheduler import OneCycleLR
import matplotlib.pyplot as plt
from torchmetrics import Accuracy

/home/shivam13juna/Documents/virtual_envs/mlo/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SyntaxError: keyword argument repeated: max_holes (datamodule.py, line 76)

<!-- Add image S9/image/README/1677323174964.png -->

![Instructions for S9](image/README/1677323174964.png)

In [ ]:

# create train_loader from torchvision datasets cifar10
batch_size=32
cifar_dm = CIFARDataModule(batch_size=batch_size)
cifar_dm.setup()

In [ ]:
# extract train_loader from cifar_dm
train_loader = cifar_dm.train_dataloader()

In [ ]:
# Extract a batch of images and labels from train_loader
images, labels = next(iter(train_loader))

In [ ]:
# n_out = (n_in - k + 2*p)/s + 1
# j_out = j_in * s
# r_out = r_in + (k - 1) * j_in
# j_in = j_out_previous, initially 1

# Output size = (Input size + 2 * padding - dilation * (kernel size - 1) - 1) / stride + 1

# n_out = (n_in + 2* p - d*(k-1) - 1)/s + 1

# so if d == 1 then n_out = (n_in + 2*p - k)/s + 1
# so if d == 2 then n_out = (n_in + 2*p - 2k + 1)/s + 1
# so if d == 3 then n_out = (n_in + 2*p - 3k + 2)/s + 1

import torch.nn as nn

class PrintShape(nn.Module):
    def __init__(self, text=None):
        super(PrintShape, self).__init__()
        self.text = text

    def forward(self, x):
        print("For the: ", self.text, " the shape is: ", x.shape)
        return x



class ULTIMUS(nn.Module):
    def __init__(self):
        super(ULTIMUS, self).__init__()
        self.key  = nn.Linear(in_features=48, out_features=8, bias=False)
        self.query = nn.Linear(in_features=48, out_features=8, bias=False)
        self.value = nn.Linear(in_features=48, out_features=8, bias=False)

        self.softmax = nn.Softmax(dim=1)

        self.out = nn.Linear(in_features=8, out_features=48, bias=False)


    def forward(self, x):

       
        key = self.key(x)
        query = self.query(x)
        value = self.value(x)

        softmax_output = self.softmax(torch.matmul(query.T, key)/torch.sqrt(torch.tensor(8.0)))

        pre_output = torch.matmul(value, softmax_output)

        output = self.out(pre_output)

        return output


class Att_Model(torch.nn.Module):
    def __init__(self):
        super(Att_Model, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            # Here n_in=32, n_out=32, k = 3, p = 1, s = 1, d = 1, j_in=1, r_out = 3

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # Here n_in=32, n_out=32, k = 3, p = 1, s = 1, d = 1, j_in=1, r_out = 5

            nn.Conv2d(in_channels=32, out_channels=48, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(48),
            nn.ReLU()
            # Here n_in=32, n_out=32, k = 3, p = 1, s = 1, d = 1, j_in=1, r_out = 7
        )

        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.ultimus_1 = ULTIMUS()
        self.ultimus_2 = ULTIMUS()
        self.ultimus_3 = ULTIMUS()
        self.ultimus_4 = ULTIMUS()

        

        self.final_layer = nn.Linear(in_features=48, out_features=10, bias=False)

        
    def forward(self, x):

        x = self.conv_layer(x)

        average_pooled = self.avg_pool(x)

        # squash the output of the average pool layer
        average_pooled = average_pooled.view(average_pooled.size(0), -1)
        
        ultimus_1_output = self.ultimus_1(average_pooled)

        ultimus_2_output = self.ultimus_2(ultimus_1_output)

        ultimus_3_output = self.ultimus_3(ultimus_2_output)

        ultimus_4_output = self.ultimus_4(ultimus_3_output)

        final_output = self.final_layer(ultimus_4_output)

        return final_output



        


# print model summary using torchsummary
# eva = Att_Model()
# eva = eva.to("cuda")
# from torchsummary import summary
# summary(eva, input_size=(3, 32, 32))

In [ ]:
class LitResnet(LightningModule):
    def __init__(self, lr=0.05):
        super().__init__()

        self.save_hyperparameters()
        self.model = Att_Model() 
        self.accuracy = Accuracy(task='multiclass', num_classes=10)

        self.ce = nn.CrossEntropyLoss()
    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        # loss = F.nll_loss(logits, y)
        loss = self.ce(logits, y)
        self.log("train_loss", loss)
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        # loss = F.nll_loss(logits, y)
        loss = self.ce(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_acc", acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr,  weight_decay=5e-4)
        
        # Use OnecycleLR
        # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.005, pct_start=0.2, div_factor=10, final_div_factor=10, steps_per_epoch=len(cifar_dm.train_dataloader()), epochs=24)

        # return [optimizer], [scheduler]
        return optimizer

In [ ]:
model = LitResnet(lr=0.005)

trainer = Trainer(
    max_epochs=30,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None, 
)

trainer.fit(model, cifar_dm)
trainer.test(model, datamodule=cifar_dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type      | Params
---------------------------------------
0 | model    | Att_Model | 25.7 K
1 | accuracy | Accuracy  | 0     
---------------------------------------
25.7 K    Trainable params
0         Non-trainable params
25.7 K    Total params
0.103     Total estimated model params size (MB)


/home/shivam13juna/Documents/virtual_envs/mlo/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/shivam13juna/Documents/virtual_envs/mlo/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 29: 100%|██████████| 1564/1564 [00:23<00:00, 66.54it/s, loss=2.3, v_num=24, val_loss=2.300, val_acc=0.101]

...

Epoch 29: 100%|██████████| 1564/1564 [00:23<00:00, 66.51it/s, loss=2.3, v_num=24, val_loss=2.300, val_acc=0.101]
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/shivam13juna/Documents/virtual_envs/mlo/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 313/313 [00:02<00:00, 136.91it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.10000000149011612    │
│         test_loss         │     2.30259108543396      │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.30259108543396, 'test_acc': 0.10000000149011612}]